In [91]:
# initialize project:
using Pkg; Pkg.activate("./MyProject");;
include("libs/local_clifford_group.jl"); include("libs/cnc.jl"); include("libs/utils.jl");

using GAP
const g = GAP.Globals;

  Activating project at `~/GitHub/LocalLambda/MyProject`


In [2]:
using JuMP

using JuMP
using GLPK
using LinearAlgebra

Generate Clifford group and local Clifford group

In [93]:
# 3-qubit local Clifford group:
LC2 = LocalCliffordGroup(2); local_cl2 = LC2.LocalCliffGroup; PS2 = PauliString(2);

In [94]:
# 3-qubit local Clifford group:
LC3 = LocalCliffordGroup(3); local_cl3 = LC3.LocalCliffGroup; PS3 = PauliString(3);

In [95]:
# 2-qubit Clifford group
C2 = CliffordGroup(2); cl2 = C2.CliffGroup;

In [4]:
# 3-qubit Clifford group
C3 = CliffordGroup(3); cl3 = C3.CliffGroup;

In [70]:
PS2 = PauliString(2);
PS3 = PauliString(3);

In [98]:
fieldnames(PauliString)

(:bit_strings, :pauli_strings, :pauli_to_bit, :bit_to_pauli, :bit_to_int, :int_to_bit)

# magic states

In [62]:
# Define magic state
T = Vector{Real}([1,1/sqrt(2),1/sqrt(2),0]);
T2 = Vector{Float64}(kron(T,T));;
T3 = Vector{Float64}(kron(kron(T,T),T));;
T4 = Vector{Float64}(kron(T3,T));;
T5 = Vector{Float64}(kron(T4,T));;
T6 = Vector{Float64}(kron(T5,T));;
T7 = Vector{Float64}(kron(T6,T));;
T8 = Vector{Float64}(kron(T7,T));;

In [122]:
T3_L1 = sum(Vector{Real}([abs(x) for x in T3]))

14.07106781186547

In [123]:
CNC1_L1 = sum(Vector{Real}([abs(x) for x in CNC1[:,1]]))

16//1

In [124]:
CNC2_L1 = sum(Vector{Real}([abs(x) for x in CNC2[:,1]]))

12//1

In [125]:
CNC3_L1 = sum(Vector{Real}([abs(x) for x in CNC3[:,1]]))

8//1

# one-qubit phase space

In [4]:
cnc_sets_1_1 = generate_all_cncs(1,[1]);
cnc_1_1 = generate_cnc_vertices(1,cnc_sets_1_1);

In [5]:
cnc_1_1

4×8 Matrix{Rational{Int64}}:
  1   1   1   1   1  1   1   1
 -1  -1   1  -1   1  1   1  -1
  1   1   1  -1  -1  1  -1  -1
  1  -1  -1  -1   1  1  -1   1

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(v1)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(v1) * x .== T)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0
Optimal value of x: [0.35355339059327373, 0.14644660940672627, 0.5]


In [12]:
c1 = cnc_1_1[:,[3,4,6]]

4×3 Matrix{Rational{Int64}}:
  1   1  1
  1  -1  1
  1  -1  1
 -1  -1  1

In [14]:
s1 = Matrix{Rational{Int64}}([1 1 1; 1 0 -1; 0 1 0; 0 0 0])

4×3 Matrix{Rational{Int64}}:
 1  1   1
 1  0  -1
 0  1   0
 0  0   0

In [19]:
S1 = Matrix{Rational{Int64}}([1 1 1 1 1; 1 0 -1 0 0; 0 1 0 -1 0; 0 0 0 0 1])

4×5 Matrix{Rational{Int64}}:
 1  1   1   1  1
 1  0  -1   0  0
 0  1   0  -1  0
 0  0   0   0  1

In [20]:
c2 = Matrix{Rational{Int64}}(undef,16,0)
for i in 1:size(c1)[2]
    for j in 1:size(S1)[2]
        v = kron(c1[:,i],S1[:,j])
        c2 = hcat(c2,v)
    end
end

In [21]:
c2

16×15 Matrix{Rational{Int64}}:
  1   1   1   1   1   1   1   1   1   1  1  1   1   1  1
  1   0  -1   0   0   1   0  -1   0   0  1  0  -1   0  0
  0   1   0  -1   0   0   1   0  -1   0  0  1   0  -1  0
  0   0   0   0   1   0   0   0   0   1  0  0   0   0  1
  1   1   1   1   1  -1  -1  -1  -1  -1  1  1   1   1  1
  1   0  -1   0   0  -1   0   1   0   0  1  0  -1   0  0
  0   1   0  -1   0   0  -1   0   1   0  0  1   0  -1  0
  0   0   0   0   1   0   0   0   0  -1  0  0   0   0  1
  1   1   1   1   1  -1  -1  -1  -1  -1  1  1   1   1  1
  1   0  -1   0   0  -1   0   1   0   0  1  0  -1   0  0
  0   1   0  -1   0   0  -1   0   1   0  0  1   0  -1  0
  0   0   0   0   1   0   0   0   0  -1  0  0   0   0  1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  1  1   1   1  1
 -1   0   1   0   0  -1   0   1   0   0  1  0  -1   0  0
  0  -1   0   1   0   0  -1   0   1   0  0  1   0  -1  0
  0   0   0   0  -1   0   0   0   0  -1  0  0   0   0  1

In [22]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c2)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c2) * x .== T2)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.414213562373095
Optimal value of x: [0.24999999999999992, 0.17677669529663687, 0.0, -0.07322330470336308, 0.0, 0.10355339059327379, 0.07322330470336313, 0.0, -0.030330085889910652, 0.0, 0.24999999999999997, 0.35355339059327373, -0.10355339059327373, 0.0, 0.0]


# two-qubit cnc phase space:

In [101]:
cnc_sets_2_2 = generate_all_cncs(2,[2]);
cnc_2_2 = generate_cnc_vertices(2,cnc_sets_2_2);

In [23]:
cnc_sets_2_1 = generate_all_cncs(2,[1]);
cnc_2_1 = generate_cnc_vertices(2,cnc_sets_2_1);

In [67]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(cnc_2_1)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(cnc_2_1) * x .== T2)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 0.9999999999999998
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06066017177982124, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04289321881345246, 0.0, 2.0816681711721685e-17, 0.0, 0.0, 0.0, 0.042893218813452545, 0.0, 0.0, 0.06066017177982119, -1.422473250300982e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14644660940672632, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14644660940672624, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.1564823173178714e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1464466094067263, 0.0, 0.0, 0.04289321881345256, 0.0, 0.0, 

In [69]:
obj_value

0.9999999999999998

In [72]:
cnc_2_1[:,non_zero_optimal_2_1]

16×12 Matrix{Rational{Int64}}:
  1   1   1   1  1  1   1   1  1   1   1  1
  0   0   0   1  1  0   1   1  1   1   1  1
  1   1   1   1  1  1   0   0  1   0   1  0
  0   0   0  -1  1  0   0   0  1   0  -1  0
  1   0   1   0  0  1   1   0  1   1   1  1
  0   1   0   0  0  0   1   0  1   1   1  1
  1   0   1   0  0  1   0   1  1   0   1  0
  0   1   0   0  0  0   0   1  1   0  -1  0
  1   1   0   1  1  1   1   1  0   0   0  1
  0   0   1   1  1  0   1   1  0   0   0  1
  1   1   0   1  1  1   0   0  0   1   0  0
  0   0  -1  -1  1  0   0   0  0  -1   0  0
 -1   0   0   0  0  1  -1   0  0   0   0  1
  0   1   1   0  0  0  -1   0  0   0   0  1
 -1   0   0   0  0  1   0  -1  0  -1   0  0
  0  -1   1   0  0  0   0   1  0  -1   0  0

In [71]:
non_zero_optimal_2_1 = findall(x->x>(10^(-5)),x_value)

for x in non_zero_optimal_2_1
    display(pauli_basis_to_cnc(cnc_2_1[:,x],PS2))
end

CNC:
Maximal CNC set:
- Isotropic generators:
  ["IY"]
- Anticommuting Paulis:
  ["YI", "XY", "ZY"]

- Value assignment:
  IY: 1
  YI: 1
  II: 1
  XY: 1
  ZY: -1
  XI: 1
  ZI: -1
  YY: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["YY"]
- Anticommuting Paulis:
  ["IY", "XX", "XZ"]

- Value assignment:
  IY: 1
  YI: 1
  II: 1
  XX: 1
  XZ: 1
  ZX: 1
  ZZ: -1
  YY: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["XY"]
- Anticommuting Paulis:
  ["YX", "YZ", "IY"]

- Value assignment:
  YX: 1
  YZ: -1
  IY: 1
  II: 1
  XY: 1
  ZX: 1
  ZZ: 1
  XI: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["YI"]
- Anticommuting Paulis:
  ["YX", "YZ", "IY"]

- Value assignment:
  YX: 1
  YZ: -1
  IY: 1
  YI: 1
  IX: 1
  IZ: -1
  II: 1
  YY: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["YI"]
- Anticommuting Paulis:
  ["YX", "YZ", "IY"]

- Value assignment:
  YX: 1
  YZ: 1
  IY: 1
  YI: 1
  IX: 1
  IZ: 1
  II: 1
  YY: 1


...

CNC:
Maximal CNC set:
- Isotropic generators:
  ["YX"]
- Anticommuting Paulis:
  ["YI", "XY", "ZY"]

- Value assignment:
  YX: 1
  YI: 1
  IX: 1
  II: 1
  XY: 1
  ZY: -1
  XZ: 1
  ZZ: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["XI"]
- Anticommuting Paulis:
  ["IY", "IX", "IZ"]

- Value assignment:
  IY: 1
  IX: 1
  IZ: 1
  II: 1
  XY: 1
  XX: 1
  XZ: 1
  XI: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["XX"]
- Anticommuting Paulis:
  ["YZ", "IX", "ZZ"]

- Value assignment:
  YZ: -1
  IX: 1
  II: 1
  ZY: -1
  XX: 1
  ZZ: -1
  XI: 1
  YY: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["XI"]
- Anticommuting Paulis:
  ["IY", "IX", "IZ"]

- Value assignment:
  IY: 1
  IX: 1
  IZ: -1
  II: 1
  XY: 1
  XX: 1
  XZ: -1
  XI: 1


CNC:
Maximal CNC set:
- Isotropic generators:
  ["IX"]
- Anticommuting Paulis:
  ["YX", "XX", "ZX"]

- Value assignment:
  YX: 1
  YI: 1
  IX: 1
  II: 1
  XX: 1
  ZX: 1
  XI: 1
  ZI: 1


In [27]:
c3 = Matrix{Rational{Int64}}(undef,64,0)
for i in 1:size(cnc_2_1)[2]
    for j in 1:size(s1)[2]
        v = kron(cnc_2_1[:,i],S1[:,j])
        c3 = hcat(c3,v)
    end
end

In [28]:
c3

64×720 Matrix{Rational{Int64}}:
 1  1   1   1   1   1   1   1   1   1  …   1  1  1   1  1  1   1   1   1   1
 1  0  -1   1   0  -1   1   0  -1   1     -1  1  0  -1  1  0  -1   1   0  -1
 0  1   0   0   1   0   0   1   0   0      0  0  1   0  0  1   0   0   1   0
 0  0   0   0   0   0   0   0   0   0      0  0  0   0  0  0   0   0   0   0
 0  0   0   0   0   0   0   0   0  -1     -1  1  1   1  0  0   0   1   1   1
 0  0   0   0   0   0   0   0   0  -1  …   1  1  0  -1  0  0   0   1   0  -1
 0  0   0   0   0   0   0   0   0   0      0  0  1   0  0  0   0   0   1   0
 0  0   0   0   0   0   0   0   0   0      0  0  0   0  0  0   0   0   0   0
 0  0   0   0   0   0   1   1   1   0     -1  0  0   0  0  0   0  -1  -1  -1
 0  0   0   0   0   0   1   0  -1   0      1  0  0   0  0  0   0  -1   0   1
 ⋮                  ⋮                  ⋱   ⋮               ⋮              
 0  0   0   0   0   0   0   0   0   0  …   0  0  0   0  0  0   0   0   0   0
 0  0   0   0   0   0  -1  -1  -1   1      0  

In [29]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c3) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.4142135623730945
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1503343087629182e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5037089277034076e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.239936510889834e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030330085889910638, 0.0, 0.0, 0.0, 0.0, 0.0, 5.059610138265689e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04105339059327375, 0.07322330470336308, -0.021446609406726203, 0.0, 0.0, -0.008883476483184362, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010723304703363115, 0.0, -1.3183898417423733e-17, 0.0, 0.0, 0.0, 0.0, 0.0, -8.782240546405799e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.4719809532669504e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04105339059327374, 0.042893218813452455, -0.021446609

In [32]:
c4 = Matrix{Rational{Int64}}(undef,64*4,0)
for i in 1:size(c3)[2]
    for j in 1:size(s1)[2]
        v = kron(c3[:,i],S1[:,j])
        c4 = hcat(c4,v)
    end
end

In [34]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c4) * x .== T4)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.9999999999999987
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.677811481187085e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.081487911019576e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015165042944955368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5

In [36]:
c5 = Matrix{Rational{Int64}}(undef,64*4*4,0)
for i in 1:size(c4)[2]
    for j in 1:size(s1)[2]
        v = kron(c4[:,i],S1[:,j])
        c5 = hcat(c5,v)
    end
end

In [37]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c5)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c5) * x .== T5)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 2.82842712474619
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.734723475976807e-17, 0.0, 0.0, -3.469446951953614e-18, 0.0, 0.0, 0.0, 0.0, 1.0959863512992716e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [39]:
c6 = Matrix{Rational{Int64}}(undef,64*4*4*4,0)
for i in 1:size(c5)[2]
    for j in 1:size(s1)[2]
        v = kron(c5[:,i],s1[:,j])
        c6 = hcat(c6,v)
    end
end

In [64]:
s5 = Matrix{Rational{Int64}}(undef,4^5,0)
for i1 in 1:3
    for i2 in 1:3
        for i3 in 1:3
            for i4 in 1:3
                for i5 in 1:3
                    v = kron(s1[:,i5],kron(s1[:,i4],kron(s1[:,i3],kron(s1[:,i2],s1[:,i1]))));
                    s5 = hcat(s5,v)
                end
            end
        end
    end
end

In [73]:
c7 = kron(cnc_2_1[:,non_zero_optimal_2_1],s5)

16384×2916 Matrix{Rational{Int64}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …   1   1   1   1   1   1   1   1   1
 1  1  1  1  1  1  1  1  1  1  1  1  1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 0  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0   0   0   0   0   0   0
 1  1  1  1  1  1  1  1  1  1  1  1  1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …   1   1   1   1   1   1   1   1   1
 0  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0   0   0   0   0   0   0
 ⋮              ⋮              ⋮        ⋱               ⋮                   ⋮
 0  0  0  0  0  0  0  0  0  0  0  0  0  …   0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0  

In [43]:
c6

4096×19440 Matrix{Rational{Int64}}:
 1  1   1  1  1   1   1   1   1  1  1  …   1  1  1   1  1  1   1   1   1   1
 1  0  -1  1  0  -1   1   0  -1  1  0     -1  1  0  -1  1  0  -1   1   0  -1
 0  1   0  0  1   0   0   1   0  0  1      0  0  1   0  0  1   0   0   1   0
 0  0   0  0  0   0   0   0   0  0  0      0  0  0   0  0  0   0   0   0   0
 1  1   1  0  0   0  -1  -1  -1  1  1     -1  1  1   1  0  0   0  -1  -1  -1
 1  0  -1  0  0   0  -1   0   1  1  0  …   1  1  0  -1  0  0   0  -1   0   1
 0  1   0  0  0   0   0  -1   0  0  1      0  0  1   0  0  0   0   0  -1   0
 0  0   0  0  0   0   0   0   0  0  0      0  0  0   0  0  0   0   0   0   0
 0  0   0  1  1   1   0   0   0  0  0      0  0  0   0  1  1   1   0   0   0
 0  0   0  1  0  -1   0   0   0  0  0      0  0  0   0  1  0  -1   0   0   0
 ⋮                ⋮                 ⋮  ⋱   ⋮               ⋮              
 0  0   0  0  0   0   0   0   0  0  0      0  0  0   0  0  0   0   0   0   0
 0  0   0  0  0   0   0   0   0  0  0     

In [44]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c6)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c6) * x .== T6)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 4.000000000000013
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.119968255444917e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [74]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c7)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c7) * x .== T7)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 5.656854249492316
Optimal value of x: [0.0018956303681194426, 0.0026808261758407524, -0.0007851958077213122, 0.0026808261758407767, 0.0037912607362388393, -0.0011104345603980585, -0.0007851958077214212, -0.0011104345603980235, 0.00032523875267665326, 0.002680826175840781, 0.0037912607362388497, -0.001110434560398073, 0.003791260736238829, 0.00536165235168156, -0.001570391615442722, -0.001110434560398008, -0.0015703916154427488, 0.0004599570550447245, -0.0007851958077213722, -0.0011104345603980758, 0.0003252387526767028, -0.0011104345603980463, -0.0015703916154427401, 0.0004599570550447122, 0.00032523875267656283, 0.0004599570550447309, -0.00013471830236808828, 0.0026808261758407264, 0.0037912607362388592, -0.001110434560398131, 0.0037912607362388497, 0.005361652351681544, -0.0015703916154427187, -0.0011104345603980296, -0.0015703916154427406, 0.00045995705504469795, 0.003791260736238859, 0.005361652351681534, -0.0015703916154426961, 0.005361652351

In [75]:
c8 = kron(kron(cnc_2_1[:,non_zero_optimal_2_1],s5),s1)

65536×8748 Matrix{Rational{Int64}}:
 1  1   1  1  1   1  1  1   1  1  1  …   1   1   1   1   1   1   1   1   1
 1  0  -1  1  0  -1  1  0  -1  1  0      1   0  -1   1   0  -1   1   0  -1
 0  1   0  0  1   0  0  1   0  0  1      0   1   0   0   1   0   0   1   0
 0  0   0  0  0   0  0  0   0  0  0      0   0   0   0   0   0   0   0   0
 1  1   1  1  1   1  1  1   1  1  1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1  0  -1  1  0  -1  1  0  -1  1  0  …  -1   0   1  -1   0   1  -1   0   1
 0  1   0  0  1   0  0  1   0  0  1      0  -1   0   0  -1   0   0  -1   0
 0  0   0  0  0   0  0  0   0  0  0      0   0   0   0   0   0   0   0   0
 0  0   0  0  0   0  0  0   0  0  0      0   0   0   0   0   0   0   0   0
 0  0   0  0  0   0  0  0   0  0  0      0   0   0   0   0   0   0   0   0
 ⋮                ⋮               ⋮  ⋱       ⋮                   ⋮      
 0  0   0  0  0   0  0  0   0  0  0      0   0   0   0   0   0   0   0   0
 0  0   0  0  0   0  0  0   0  0  0      0   0   0   0   0   0   0

In [76]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c8)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c8) * x .== T8)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 7.999999999999593
Optimal value of x: [0.000947815184059683, 0.0013404130879204174, -0.00039259790386072, 0.0013404130879204356, 0.0018956303681193988, -0.0005552172801990273, -0.0003925979038607052, -0.0005552172801989903, 0.0001626193763383681, 0.001340413087920414, 0.0018956303681194023, -0.000555217280199012, 0.0018956303681193806, 0.002680826175840804, -0.0007851958077213321, -0.00055521728019902, -0.0007851958077213651, 0.00022997852752228268, -0.00039259790386066135, -0.0005552172801990439, 0.00016261937633834658, -0.0005552172801990072, -0.0007851958077213706, 0.00022997852752230756, 0.00016261937633834322, 0.00022997852752232287, -6.735915118391209e-5, 0.001340413087920404, 0.0018956303681194055, -0.0005552172801989968, 0.0018956303681193793, 0.002680826175840815, -0.0007851958077213842, -0.0005552172801990207, -0.0007851958077213821, 0.00022997852752232735, 0.0018956303681193951, 0.0026808261758408036, -0.0007851958077213592, 0.002680826

In [81]:
using JLD2

@save "./data/x_value_8.jld" x_value
@save "./data/cnc_vectors_subset_8.jld" c8

In [82]:
load("./data/cnc_vectors_subset_8.jld")

Dict{String, Any} with 1 entry:
  "c8" => Rational{Int64}[1 1 … 1 1; 1 0 … 0 -1; … ; 0 0 … 0 0; 0 0 … 0 0]

In [90]:
x_value

8748-element Vector{Float64}:
  0.000947815184059683
  0.0013404130879204174
 -0.00039259790386072
  0.0013404130879204356
  0.0018956303681193988
 -0.0005552172801990273
 -0.0003925979038607052
 -0.0005552172801989903
  0.0001626193763383681
  0.001340413087920414
  ⋮
  2.7901073970389156e-5
  3.945807721364409e-5
 -1.1557003243304359e-5
  3.945807721366049e-5
  5.580214794067293e-5
 -1.6344070727037892e-5
 -1.1557003243309046e-5
 -1.6344070727013823e-5
  4.787067483695085e-6

In [87]:
findall(x->abs(x)>10^-6,x_value)

8748-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
    ⋮
 8740
 8741
 8742
 8743
 8744
 8745
 8746
 8747
 8748

# Three-qubit CNC phase space

In [30]:
cncs1 = generate_all_cncs(3,[1]);
CNC1 = generate_cnc_vertices(3,cncs1);

In [31]:
CNC1

64×10080 Matrix{Rational{Int64}}:
  1  1   1   1   1   1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
  0  1   0   0   0   0   0   0   0   0      0   0   0   0  -1   1  -1   0   0
 -1  0   0   0   0   0   0   0   0   0      1   0   0   1   0   0   0   0  -1
  0  0   0   0   0  -1   0   1   1   0      0   0   0   0   0   0   0   1   0
  0  0   0   0   1   0   0   0   0   0      0   0   0   1   0   0   0   0   0
  0  0  -1   0   0   1  -1   0  -1  -1  …   0   0   1   0   0   0   0   0   0
  0  0   0   0   0  -1   0   0   1   0      0   0   0   1   0   0   1   0   0
  0  0   0  -1   0   0   0   0   0   0      0  -1   0   0   0   0  -1   0   0
  0  0   0   0   0   1  -1   0   0  -1      0   0   0   0   0   0   0  -1   0
  1  0   0  -1   1   0   0   0   0   0     -1   0   1   0   0   0   0   0   0
  ⋮                  ⋮                  ⋱                   ⋮              
  0  1   0   0  -1   0   1   0   0   0  …   0   1   1   0   0   0   0   0   0
  0  0   0   0   0   0   0   0  

In [113]:
cncs2 = generate_all_cncs(3,[2]);
CNC2 = generate_cnc_vertices(3,cncs2);

In [11]:
CNC2

64×24192 Matrix{Rational{Int64}}:
  1   1   1  1   1   1   1   1  1   1  …   1   1   1   1   1   1   1   1   1
  0   0   0  1   0  -1   0   0  0   0     -1  -1   0  -1   0  -1   0   0   0
  1  -1   0  0   0   1   0   0  0   1      0   1   0   0   0   0   0   0   0
  0   0   0  0   0   0   0   0  0   0      1   0   0   0   0   0   0   0   0
  0   0   0  0   0   0  -1   0  0   0      0   0  -1   0   1   0   0   0   1
 -1   1   1  0   0   0   0   0  0  -1  …   0   0   0   0   0   0   0  -1   0
  0   0   0  0   0   0   0   1  0   0      0   0   0  -1   0   0  -1   0   0
  0   0   0  0   0   0   0   0  0   0      0   0   0   0   0   0   0  -1   0
  0   0   0  0   0   0   0   0  0   0      0   0   0   0   0   0   0   0   0
  1   1   0  0   0   0   0  -1  0   0      0   0   0   0   0   0   0   0   0
  ⋮                  ⋮                 ⋱           ⋮                   ⋮  
  1   0  -1  0   0   0  -1   0  0   0  …   0   0   0   1   0  -1   0   0   0
  0   0  -1  0   0   0   0   0  0   0      0

In [114]:
cncs3 = generate_all_cncs(3,[3]);
CNC3 = generate_cnc_vertices(3,cncs3);

In [12]:
CNC3

64×36864 Matrix{Rational{Int64}}:
 1   1  1  1  1   1   1   1   1   1  1  …  1  1   1   1   1   1   1   1  1  1
 1   0  0  0  0   0   0   0   0   0  0     0  0   0   0   0   0   0   0  0  0
 0   0  0  0  0  -1   0   0   0   0  0     0  0   0  -1   0   0   0   0  1  0
 0   0  0  1  0   1   0   0   0   0  0     0  0   0   0   0   0   0   0  1  0
 0   0  0  0  0   0   0   0   0   0  0     0  0   1   0   0   0   0   0  0  0
 0   0  0  1  0   0   0  -1  -1   1  0  …  0  0   0   0   0   0   0   0  0  0
 0   0  0  0  0   0   0   0   0   0  0     0  0   0   0   0   0  -1   0  0  0
 0   0  0  0  0   0   0  -1   0   0  0     0  0   0   0   0   0   0   1  0  0
 0   0  0  0  0   0  -1  -1   0   0  1     0  0   0   0   0   0  -1   0  0  0
 0   0  0  0  0  -1   0   0   0   0  0     0  0   0   0   0   0   0   0  0  0
 ⋮                ⋮                  ⋮  ⋱     ⋮                   ⋮         
 0   0  0  0  0   0   0   0   1   0  0  …  0  0   0  -1   0   0  -1   0  0  0
 0   0  0  0  0   0   0   0   1

In [7]:
cncs = generate_all_cncs(3,[1,2,3]);
CNC = generate_cnc_vertices(3,cncs);

In [11]:
CNC

64×71136 Matrix{Rational{Int64}}:
  1  1   1   1   1   1  1   1  1  1  …   1  1   1  1   1  1   1   1   1   1
  1  0   0   0   0   0  0   0  0  0      0  0   0  0   0  0   0   0   0   1
  0  0   0   0   0   0  0   0  0  0      0  0   0  0   0  0   1   0   0   1
 -1  0   0   0   0   0  0   0  0  0      0  0   0  0   0  0   1   1   0   0
  0  0   0   0   0   0  0   0  0  0      0  0   0  0   0  0   0   0   0   0
  0  0   0   0   0   0  0   0  0  0  …   0  0   0  0   0  1  -1   0   0   0
  0  0   0   0   0   0  0   0  0  0     -1  0   0  0   0  0   0   0   0   0
  0  0   0   1   0   0  0   0  0  0      0  0  -1  0   0  1   0   0   0   0
  0  0   0   0   0   0  0  -1  0  0      0  0   0  0   0  0   0   0   0   0
  0  0   0   1   0  -1  0   0  0  0      0  0   0  0   0  0  -1   0   0   0
  ⋮                  ⋮               ⋱                 ⋮                  ⋮
  0  0   0   1  -1   0  0   0  0  0  …   0  1  -1  0   0  0   0   0  -1   0
  0  0   0   0  -1   0  0   0  1  0      1  0   0  0  

In [8]:
s3 = generate_all_cncs(3,[0]);
stab3 = generate_cnc_vertices(3,s3);

In [ ]:
for i in 1:3    
    cncs_i = generate_all_cncs(3,[i]);
    CNCi = generate_cnc_vertices(3,cncs_i);
    println(size(CNCi))
end

In [178]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(CNC1[:,non_zero_optimal_3_1])[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(CNC1[:,non_zero_optimal_3_1]) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [179]:
obj_value

1.282842712474618

In [170]:
x_abs = Vector{Real}([abs(x) for x in x_value])
#x_min = minimum(x_abs[findall(x->abs(x)>0,x_value)])
x_min = 10^(-15);;

In [172]:
non_zero_optimal_3_1 = findall(x->x>x_min,x_abs)

36-element Vector{Int64}:
  339
  764
 1202
 1351
 1896
 2047
 2272
 2289
 2648
 3176
    ⋮
 6666
 7132
 8366
 8440
 8533
 9137
 9138
 9252
 9384

In [116]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(CNC2)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(CNC2) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Excessive output truncated after 524288 bytes.

Optimal objective value (min ||x||_1): 2.3165824894352776
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.0740661672321725e-17, 0.0, 0.0, 0.0, 0.0, 0

In [117]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(CNC3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(CNC3) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [119]:
obj_value

3.974873734152914

# four-qubit cnc

In [ ]:
using JLD2

cnc_4_1_orbit = load("./data/cnc_orbit_4_1.jld")["cnc_orbit_4_1"]

Set{Set{Vector{Int64}}} with 9703 elements:
  Set([[0, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 1, 1, 0], [1, 0, 0, 0, 1, 0, 1…
  Set([[0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 1, 1, 0, 1], [0, 1, 1, 1, 0, 0, 0…
  Set([[1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1, 1…
  Set([[1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 1, 1], [1, 1, 1, 1, 1, 0, 0…
  Set([[0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1, 1…
  Set([[1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 1, 0, 1…
  Set([[1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1…
  Set([[0, 1, 1, 1, 0, 0, 0, 1], [0, 1, 0, 1, 1, 1, 1, 0], [1, 0, 0, 0, 0, 0, 0…
  Set([[0, 0, 0, 1, 1, 0, 0, 1], [1, 0, 0, 0, 1, 0, 1, 0], [1, 1, 1, 1, 0, 1, 0…
  Set([[1, 1, 0, 0, 0, 1, 0, 1], [1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0…
  Set([[1, 0, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 0, 1, 1…
  Set([[1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0, 1, 0], [0, 1,

In [7]:
using JLD2
cnc_vectors_4_1 = load("./cnc_vectors_4_1.jld")["cnc_vectors_4_1"]

256×731328 Matrix{Rational{Int64}}:
 1  1   1   1   1  1   1  1  1   1  1  1  …   1   1   1   1   1   1  1  1   1
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      1  -1  -1  -1  -1  -1  1  1  -1
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0  …   0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 ⋮                 ⋮                ⋮     ⋱       ⋮                  ⋮     
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 1  1  -1  -1  -1  1  -1  1  1

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(cnc_vectors_4_1)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(cnc_vectors_4_1) * x .== T4)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

# three-qubit LC phase space

In [ ]:
d = Vector{Int}([1 for i in 1:64]);
D = array_to_matrix(local_clifford_orbit_of_point(LC3,d)); 

In [207]:
V = Matrix{Rational{Int64}}(hcat(D,CNC))

64×71648 Matrix{Rational{Int64}}:
 1  1   1   1  1   1   1   1   1   1  …   1   1   1   1   1  1   1   1   1
 1  1   1   1  1   1   1   1   1  -1      0  -1   0   0   1  0   0   1   0
 1  1   1  -1  1   1  -1   1  -1  -1      1   0   1   0   0  0   0   0   0
 1  1   1   1  1   1   1   1   1   1      0   0   0   0   0  0   0   0   0
 1  1   1   1  1   1   1  -1   1   1      0   0   0   0   0  0   0   0  -1
 1  1   1   1  1   1   1  -1   1  -1  …   0   0   0   0   0  0   0   0   0
 1  1   1  -1  1   1  -1  -1  -1  -1      0   0   0   0   0  0   0  -1   0
 1  1   1   1  1   1   1  -1   1   1      0   0   0   0   0  0   0   0   0
 1  1  -1   1  1  -1   1  -1  -1   1      0   0   1   0   0  0   0   0   0
 1  1  -1   1  1  -1   1  -1  -1  -1      0   0   0   0   0  0   0   0   0
 ⋮                 ⋮                  ⋱       ⋮                  ⋮      
 1  1   1   1  1   1   1  -1   1   1  …  -1   0   0   0   0  0   1   1   0
 1  1  -1   1  1  -1   1  -1  -1   1     -1   0  -1   0   0  0  -1  

In [197]:
# Define controlled Z gate:
for g in C3.NormalizerGates.Gens; println(g); end

GAP: [ [ 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0 ], [ 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0 ], [ 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 1/2*E(8)-1/2*E(8)^3 ], [ 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3, 0, 0 ], [ 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3, 0 ], [ 0, 0, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, -1/2*E(8)+1/2*E(8)^3 ] ]
GAP: [ [ 1, 0, 0, 0, 0, 0, 0, 0 ], [ 0, 1, 0, 0, 0, 0, 0, 0 ], [ 0, 0, 1, 0, 0, 0, 0, 0 ], [ 0, 0, 0, 1, 0, 0, 0, 0 ], [ 0, 0, 0, 0, E(4), 0, 0, 0 ], [ 0, 0, 0, 0, 0, E(4), 0, 0 ], [ 0, 0, 0, 0, 0, 0, E(4), 0 ], [ 0, 0, 0, 0, 0, 0, 0, E(4) ] ]
GAP: [ [ 1/2*E(8)-1/2*E(8)^3, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, 1/2*E(8)-1/2*E(8)^3, 0, 0, 0, 0 ], [ 1/2*E(8)-1/2*E(8)^3, 0, -1/2*E(8)+1/2*E(8)^3, 0, 0, 0, 0, 0 ], [ 0, 1/2*E(8)-1/2*E(8)^3, 0, -1/

In [198]:
# Unitaries:
CX12 = cnot(1,2,3); CX13 = cnot(1,3,3); CX23 = cnot(2,3,3);
H1 = hadamard_j(1,3); H2 = hadamard_j(2,3); H3 = hadamard_j(3,3);

In [199]:
cx12 = C3.GateGroupHom(CX12); cx13 = C3.GateGroupHom(CX13); cx23 = C3.GateGroupHom(CX23);
h1 = C3.GateGroupHom(H1); h2 = C3.GateGroupHom(H1); h3 = C3.GateGroupHom(H1); 
cz12 = h2*cx12*g.Inverse(h2); cz13 = h3*cx13*g.Inverse(h3); cz23 = h3*cx23*g.Inverse(h3);

In [200]:
function clifford_gate_action_by_gens(gens::Vector{GapObj},CG::CliffordGroup,C::Vector)
    C = Vector{Real}(C)
    fdict = CG.IntActionDict
    bdict = CG.ActionIntDict
    G = CG.CliffGroup.Grp
    N = length(C);
    K = g.Set(jlg([]))

    gg = gens[1];
    for elem in gens[2:end]; gg = elem*gg; end;

    vals, V, map = grouping(C)

    K = Vector{Vector{Int}}([])
    for v in V
        k = Vector{Int}([]);
        for e in v
            x = e; y = fdict[e]; gy = g.OnPoints(jlg(y),gg); gx = bdict[gy];
            #println("$x, $y, $gy, $gx")
            push!(k,gx); 
        end
        push!(K,k)
    end

    return subset_to_vector(N,K,map)
end

clifford_gate_action_by_gens (generic function with 1 method)

These are magic graph states where the graphs are the line $L_3$ and the complete graph $K_3$.

In [201]:
L3 = clifford_gate_action_by_gens([cz12,cz23],C3,T3);
K3 = clifford_gate_action_by_gens([cz12,cz23,cz13],C3,T3);

These are magic graph states where the initial magic state $|T\rangle^{\otimes 3}$ is acted on by a unitary $\bar H^U$, which applies $H$ to qubit $i$ if $i\in U$, where $U$ is a subset of the vertex set of $G$.

In [205]:
L3 = clifford_gate_action_by_gens([cz12,cz23],C3,T3);
H1L3 = clifford_gate_action_by_gens([h1,cz12,cz23],C3,T3);
H2L3 = clifford_gate_action_by_gens([h2,cz12,cz23],C3,T3);
H3L3 = clifford_gate_action_by_gens([h3,cz12,cz23],C3,T3);
H12L3 = clifford_gate_action_by_gens([h1,h2,cz12,cz23],C3,T3);
H13L3 = clifford_gate_action_by_gens([h1,h3,cz12,cz23],C3,T3);
H23L3 = clifford_gate_action_by_gens([h2,h3,cz12,cz23],C3,T3);
H123L3 = clifford_gate_action_by_gens([h1,h2,h3,cz12,cz23],C3,T3);

In [220]:
K3 = clifford_gate_action_by_gens([cz12,cz23,cz13],C3,T3);
H1K3 = clifford_gate_action_by_gens([h1,cz12,cz23,cz13],C3,T3);
H2K3 = clifford_gate_action_by_gens([h2,cz12,cz23,cz13],C3,T3);
H3K3 = clifford_gate_action_by_gens([h3,cz12,cz23,cz13],C3,T3);
H12K3 = clifford_gate_action_by_gens([h1,h2,cz12,cz23,cz13],C3,T3);
H13K3 = clifford_gate_action_by_gens([h1,h3,cz12,cz23,cz13],C3,T3);
H23K3 = clifford_gate_action_by_gens([h2,h3,cz12,cz23,cz13],C3,T3);
H123K3 = clifford_gate_action_by_gens([h1,h2,h3,cz12,cz23,cz13],C3,T3);

In [208]:
using JuMP

using JuMP
using GLPK
using LinearAlgebra

## Deterministic phase space

In [209]:
# Dimensions
d = size(D)[1]; N = size(D)[2];

#### $L_3$

In [214]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2071067811865475
Optimal value of x: [0.04040291308792027, 0.014235789365879045, 0.0, 0.07396670756662425, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009617539877458567, 0.0, 0.0, -0.018398751533748328, 0.0, 0.01839875153374836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03180755214744592, 0.0, 0.0, 0.014514565439601848, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07710749079750959, 2.4142788038215285e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0037912607362388323, 0.0, -2.1933486736489945e-19, -0.0014774662575349525, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003977111452054145, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07405963292453197, 0.0, 0.0, 0.0, 0.10123959611457004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0019420930470733268, 0.0, -0.02052669529663709, 0.0, 0.0, 0.0, 0.03078537321046166, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

#### $H_1 \cdot L_3$

In [215]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H1L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2071067811865477
Optimal value of x: [0.025058245739807777, 0.06522573227663017, 0.0, 0.0, 0.0, 0.0, 0.058727339979741326, 0.0, 0.0, 0.0, 0.0, 0.0, -0.008629449216106212, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015759718302367973, 0.0, 3.0814879110195794e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0025275071574925464, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013467637695973415, 0.0, -0.02904773159518393, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020833333333333367, 0.0, 0.004621362644749552, 0.0, 0.005686891104358205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07051617314271338, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08544578953657042, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.877907531704922e-17, 0.016428796523701664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020201456543960192, 0.0, 0.0, 0.0, 0.03493284781867989, 0.0, 0.0, 0.0, 0.020399681663097705, 0.0, 0.0, 0.06860194205861363, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.742068357630813e-18, 0.0, -8.015087564732002e-18, 0.0, 0.0, 0.

#### $H_{2}\cdot L_3$

In [216]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H2L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2071067811865477
Optimal value of x: [0.025058245739807777, 0.06522573227663017, 0.0, 0.0, 0.0, 0.0, 0.058727339979741326, 0.0, 0.0, 0.0, 0.0, 0.0, -0.008629449216106212, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015759718302367973, 0.0, 3.0814879110195794e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0025275071574925464, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013467637695973415, 0.0, -0.02904773159518393, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020833333333333367, 0.0, 0.004621362644749552, 0.0, 0.005686891104358205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07051617314271338, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08544578953657042, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.877907531704922e-17, 0.016428796523701664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020201456543960192, 0.0, 0.0, 0.0, 0.03493284781867989, 0.0, 0.0, 0.0, 0.020399681663097705, 0.0, 0.0, 0.06860194205861363, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.742068357630813e-18, 0.0, -8.015087564732002e-18, 0.0, 0.0, 0.

#### $H_{12}\cdot L_3$

In [217]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H12L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2071067811865475
Optimal value of x: [0.04040291308792027, 0.014235789365879045, 0.0, 0.07396670756662425, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009617539877458567, 0.0, 0.0, -0.018398751533748328, 0.0, 0.01839875153374836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03180755214744592, 0.0, 0.0, 0.014514565439601848, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07710749079750959, 2.4142788038215285e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0037912607362388323, 0.0, -2.1933486736489945e-19, -0.0014774662575349525, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003977111452054145, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07405963292453197, 0.0, 0.0, 0.0, 0.10123959611457004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0019420930470733268, 0.0, -0.02052669529663709, 0.0, 0.0, 0.0, 0.03078537321046166, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

#### $H_{13}\cdot L_3$

In [218]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H13L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2071067811865475
Optimal value of x: [0.04040291308792027, 0.014235789365879045, 0.0, 0.07396670756662425, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009617539877458567, 0.0, 0.0, -0.018398751533748328, 0.0, 0.01839875153374836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03180755214744592, 0.0, 0.0, 0.014514565439601848, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07710749079750959, 2.4142788038215285e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0037912607362388323, 0.0, -2.1933486736489945e-19, -0.0014774662575349525, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003977111452054145, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07405963292453197, 0.0, 0.0, 0.0, 0.10123959611457004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0019420930470733268, 0.0, -0.02052669529663709, 0.0, 0.0, 0.0, 0.03078537321046166, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

#### $H_{123}\cdot L_3$

In [219]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H123L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2071067811865477
Optimal value of x: [0.025058245739807777, 0.06522573227663017, 0.0, 0.0, 0.0, 0.0, 0.058727339979741326, 0.0, 0.0, 0.0, 0.0, 0.0, -0.008629449216106212, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015759718302367973, 0.0, 3.0814879110195794e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0025275071574925464, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013467637695973415, 0.0, -0.02904773159518393, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020833333333333367, 0.0, 0.004621362644749552, 0.0, 0.005686891104358205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07051617314271338, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08544578953657042, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.877907531704922e-17, 0.016428796523701664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020201456543960192, 0.0, 0.0, 0.0, 0.03493284781867989, 0.0, 0.0, 0.0, 0.020399681663097705, 0.0, 0.0, 0.06860194205861363, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.742068357630813e-18, 0.0, -8.015087564732002e-18, 0.0, 0.0, 0.

#### $K_3$

In [222]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2437184335382283
Optimal value of x: [0.06962979652347648, 0.0033551983639886858, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.01000837525566919, 0.05051378067478688, 0.0, 0.0, 0.007072576687178831, 5.679820413278867e-18, 0.0, -0.0041034529651856715, 0.0, 0.02328140388552655, 0.0, 0.0, 0.04851204192244812, -1.5462574056523142e-16, 0.0, 0.0, 0.0, 0.018815770961139594, 0.0, 0.0, 0.0, 0.0, 0.0, 0.040715105316867214, 0.0, 0.0, 0.0, 0.0, 0.0, 0.050575715746438515, 0.0, 0.0, 0.06621737832318994, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0013487443762958615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012219813905756815, 0.0, -1.454569393487323e-16, 0.0, 0.02193265950923065, 0.0, 0.0, 0.030468420245300363, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05081402556233654, 0.0, 0.0, 0.0, 0.0, -0.025259247955070434, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.3877787807814289e-17, 0.0, -4.163336342344337e-17, -0.0021303241309959826, 0.0, 0.0, 0.0, 0.0, 0

#### $H_{1}\cdot K_3$

In [221]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H1K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2437184335382265
Optimal value of x: [0.0, 0.09628978713851623, 0.0, 0.0, 0.06797217218089269, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06500294719544544, -9.771153589867457e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 3.481680463097702e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.007975119376338274, 0.0, 0.035433858640099354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0276060990360428, 0.0, 0.0027481853270246133, 0.0, 0.0, 0.0, 0.0, 0.0, -0.027998696939903343, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034353943287814825, 0.0, 0.0, 0.047948594617327776, 0.0, -9.076489208527264e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009226592974061167, 0.0, 0.0, -2.924314947773091e-17, 0.0, 0.0, 0.0, 0.0, 0.0, -1.6108816878163525e-16, 0.0, 4.111236425867849e-19, 0.0, 0.0, 0.00289554509930638, 0.010011788781782478, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03894304130168899, 0.0, 0.0029692249854471426, 0.0, 0.0, 0.0, -0.012981013767229599, 0.0, -0.006183688102473373, 0.01290733

In [223]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H12K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2437184335382283
Optimal value of x: [0.06962979652347648, 0.0033551983639886858, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.01000837525566919, 0.05051378067478688, 0.0, 0.0, 0.007072576687178831, 5.679820413278867e-18, 0.0, -0.0041034529651856715, 0.0, 0.02328140388552655, 0.0, 0.0, 0.04851204192244812, -1.5462574056523142e-16, 0.0, 0.0, 0.0, 0.018815770961139594, 0.0, 0.0, 0.0, 0.0, 0.0, 0.040715105316867214, 0.0, 0.0, 0.0, 0.0, 0.0, 0.050575715746438515, 0.0, 0.0, 0.06621737832318994, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0013487443762958615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012219813905756815, 0.0, -1.454569393487323e-16, 0.0, 0.02193265950923065, 0.0, 0.0, 0.030468420245300363, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05081402556233654, 0.0, 0.0, 0.0, 0.0, -0.025259247955070434, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.3877787807814289e-17, 0.0, -4.163336342344337e-17, -0.0021303241309959826, 0.0, 0.0, 0.0, 0.0, 0

In [224]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H123K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.2437184335382265
Optimal value of x: [0.0, 0.09628978713851623, 0.0, 0.0, 0.06797217218089269, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06500294719544544, -9.771153589867457e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 3.481680463097702e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.007975119376338274, 0.0, 0.035433858640099354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0276060990360428, 0.0, 0.0027481853270246133, 0.0, 0.0, 0.0, 0.0, 0.0, -0.027998696939903343, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034353943287814825, 0.0, 0.0, 0.047948594617327776, 0.0, -9.076489208527264e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009226592974061167, 0.0, 0.0, -2.924314947773091e-17, 0.0, 0.0, 0.0, 0.0, 0.0, -1.6108816878163525e-16, 0.0, 4.111236425867849e-19, 0.0, 0.0, 0.00289554509930638, 0.010011788781782478, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03894304130168899, 0.0, 0.0029692249854471426, 0.0, 0.0, 0.0, -0.012981013767229599, 0.0, -0.006183688102473373, 0.01290733

## Deterministic plus CNC

In [15]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, V * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0403731957466686
Optimal value of x: [0.02336119279320314, 0.027151925075876063, 0.0, 0.06487196481408994, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00463892866706392, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00602388444666345, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025934970834062092, 0.0, 0.0, 0.013414895202098437, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0589520244214108, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.674211460494356e-17, 0.009266102744499315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.007654767651741832, 0.0, 0.001838771313047063, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.035771644662753586, 0.0, 0.0, 0.0, 0.08073107100954251, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0011457649696815508, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04911072779142205, 0.

In [16]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, V * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0522970773009197
Optimal value of x: [0.02902913087920384, 0.0, 0.0, 0.042623782208716404, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03138009550107063, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06328057300642415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005203820042826781, 0.0, 0.0, 0.0, 0.0, 0.027198548261619812, 0.0, 0.0, 0.0, 2.2647778714141648e-17, 0.0, 0.022236428015745022, 0.0, 0.0, 0.05830920715795437, 0.0, 0.0, 0.0, -0.0020907736197253125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0015703916154427991, 0.0, 0.006801948466053634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009933486094344454, 0.010193677096485818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000789818609018772, 0.0, 0.0036611652351681767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03426068772981487, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007582521472477563, 0.0, 0.0, 0.0, 0.0, 0.0, 0.011243686707645768, 0.0, 0.0, 0.0, 0.0, 0.0, 0

Excessive output truncated after 524290 bytes.

 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

## CNC

In [17]:
using JuMP
using GLPK
using LinearAlgebra

# Dimensions
d = size(V)[1]; N = size(CNC)[2];

Check that we recover correct value for CNC polytope.

In [18]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, CNC * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

InterruptException: InterruptException:

In [18]:
obj_value

1.0522970773009197

#### LC phase space with deterministic vertices and tensored $2$-qubit vertices

In [225]:
using Polymake
const pm = Polymake;

include("libs/symmetries.jl");

Z = Vector{Rational{Int64}}([1,0,0,1])
ZZ = kron(Z,Z);
ZZZ = kron(Z,ZZ);

# Local 
S2 = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC2,ZZ)));
#S3 = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC3,ZZZ)));

16×36 Matrix{Rational{Int64}}:
 1   1   1   1   1  1   1   1  1   1   1  …   1   1  1   1   1  1   1   1   1
 0   0   0   0  -1  1  -1   1  0   0   0      0  -1  1  -1   1  0   0   0   0
 0   0   0   0   0  0   0   0  1  -1   1      0   0  0   0   0  1  -1   1  -1
 1  -1   1  -1   0  0   0   0  0   0   0     -1   0  0   0   0  0   0   0   0
 0   0   0   0   0  0   0   0  0   0   0      0   0  0   0   0  0   0   0   0
 0   0   0   0   0  0   0   0  0   0   0  …   0   0  0   0   0  0   0   0   0
 0   0   0   0   0  0   0   0  0   0   0      0   0  0   0   0  0   0   0   0
 0   0   0   0   0  0   0   0  0   0   0      0   0  0   0   0  0   0   0   0
 0   0   0   0   0  0   0   0  0   0   0     -1   1  1  -1  -1  1   1  -1  -1
 0   0   0   0   0  0   0   0  0   0   0      0  -1  1   1  -1  0   0   0   0
 0   0   0   0   0  0   0   0  0   0   0  …   0   0  0   0   0  1  -1  -1   1
 0   0   0   0   0  0   0   0  0   0   0      1   0  0   0   0  0   0   0   0
 1   1  -1  -1   1  1  -1  -1  1 

In [226]:
LL2 = pm.polytope.Polytope(INEQUALITIES = transpose(S2))

type: Polytope<Rational>

INEQUALITIES
  1   0   0   1  0  0  0  0   0   0   0  0   1   0   0   1
  1   0   0  -1  0  0  0  0   0   0   0  0   1   0   0  -1
  1   0   0   1  0  0  0  0   0   0   0  0  -1   0   0  -1
  1   0   0  -1  0  0  0  0   0   0   0  0  -1   0   0   1
  1  -1   0   0  0  0  0  0   0   0   0  0   1  -1   0   0
  1   1   0   0  0  0  0  0   0   0   0  0   1   1   0   0
  1  -1   0   0  0  0  0  0   0   0   0  0  -1   1   0   0
  1   1   0   0  0  0  0  0   0   0   0  0  -1  -1   0   0
  1   0   1   0  0  0  0  0   0   0   0  0   1   0   1   0
  1   0  -1   0  0  0  0  0   0   0   0  0   1   0  -1   0
  ⋮                 ⋮                 ⋮                  ⋮
  1  -1   0   0  0  0  0  0   1  -1   0  0   0   0   0   0
  1   1   0   0  0  0  0  0   1   1   0  0   0   0   0   0
  1  -1   0   0  0  0  0  0  -1   1   0  0   0   0   0   0
  1   1   0   0  0  0  0  0  -1  -1   0  0   0   0   0   0
  1   0   1   0  0  0  0  0   1   0   1  0   0   0   0   0
  1   0  -1   0  0  0  0  0   1   0  -1  0   0   0   0   0
  1   0   1   0  0  0  0  0  -1   0  -1  0   0   0   0   0
  1   0  -1   0  0  0  0  0  -1   0   1  0   0   0   0   0
  1   0   0   0  0  0  0  0   0   0   0  0   0   0   0   0

In [227]:
R2 = transpose(Matrix{Rational{Int64}}(representative_vertices(LL2)));
tR3 = kron(R2,Vector{Rational{Int64}}([1,1,1,1]));

In [231]:
R3 = Matrix{Rational{Int64}}(undef,64,0)
for k in 1:size(R2)[2]
    Vk = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC3,tR3[:,k])))
    println(size(Vk))
    R3 = hcat(R3,Vk)
end

(64, 512)
(64, 13824)
(64, 6912)
(64, 3456)
(64, 2304)


#### $K_3$

In [26]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [27]:
obj_value

0.9999999999999997

#### $H_1\cdot K_3$

In [232]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H1K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005780760351833422, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04128163014969906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001186224750022822, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03750653616750061, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000919219451244218, 0.0, 0.009514308628308123, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01175673209316684, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003498702821065356, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030831066992550293, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08947705353594672, 0.0, 0.0, 0.0, 0.01627565050606905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

#### $H_{12}\cdot K_3$

In [233]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H12K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 0.9999999999999997
Optimal value of x: [0.0, 0.0031820480453635404, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004995066154529114, 0.0, 0.030945263349900488, 0.0, 0.0, 0.0, 0.0, 0.003855000652555499, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003724305369373715, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03907006418105438, 0.0, 0.09526380934152807, 0.0, 0.0, 0.0, 0.0, 0.007066670424827372, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014153382328916327, 0.0, 0.0, 0.052275764384906394, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

#### $H_{123}\cdot K_3$

In [234]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H123K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005780760351833422, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04128163014969906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001186224750022822, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03750653616750061, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000919219451244218, 0.0, 0.009514308628308123, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01175673209316684, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003498702821065356, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030831066992550293, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08947705353594672, 0.0, 0.0, 0.0, 0.01627565050606905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

#### $L_3$

In [24]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [25]:
obj_value

1.0425339700071377

In [235]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H1L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.0425339700071374
Optimal value of x: [0.0, 0.0, 0.012852441452457336, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6653345369377347e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0010462492834792606, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013956968734634891, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003782932874864601, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0037408524296694773, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008078318085809985, 0.0, 0.0, 0.0, 0.06699877096823015, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010142443216955482, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06180101

Excessive output truncated after 524289 bytes.

0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [236]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H12L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [237]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H123L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [238]:
obj_value

1.0425339700071374

# $4$-qubit LC robustness

In [ ]:
# 3-qubit local Clifford group:
LC4 = LocalCliffordGroup(4); local_cl4 = LC4.LocalCliffGroup; PS4 = PauliString(4);

In [ ]:
tR4 = kron(R2,Vector{Rational{Int64}}([1 for i in 1:16]));

In [ ]:
T4 = kron(T,T3);

In [ ]:
# Unitaries:
CX12 = cnot(1,2,4); CX13 = cnot(1,3,4); CX23 = cnot(2,3,4); CX34 = cnot(3,4,4); CX24 = cnot(2,4,4); CX14 = cnot(1,4,4);
H1 = hadamard_j(1,4); H2 = hadamard_j(2,4); H3 = hadamard_j(3,4);  H4 = hadamard_j(4,4);

cx12 = C3.GateGroupHom(CX12); cx13 = C3.GateGroupHom(CX13); cx23 = C3.GateGroupHom(CX23);
h1 = C3.GateGroupHom(H1); h2 = C3.GateGroupHom(H1); h3 = C3.GateGroupHom(H1); 
cz12 = h2*cx12*g.Inverse(h2); cz13 = h3*cx13*g.Inverse(h3); cz23 = h3*cx23*g.Inverse(h3);

In [ ]:
R4 = Matrix{Rational{Int64}}(undef,4^4,0)
for k in 1:size(tR4)[2]
    Vk = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC4,tR4[:,k])))
    println(size(Vk))
    R4 = hcat(R4,Vk)
end

In [ ]:
using Serialization

# Save the group object to a file
open("large_group.jld", "w") do io
    serialize(io, LC4)
end

In [ ]:
using JLD2

# Save the matrix to a file
@save "R4.jld2" R4

In [28]:
R3

64×27008 Matrix{Rational{Int64}}:
  1   1   1   1   1  1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
  1   1   1   1   1  1  -1   1   1   1      0   0   0   0   0   0   0   0   0
  1   1   1   1   1  1   1  -1   1   1      0   0   0   0   0   0   0   0   0
  1   1   1   1   1  1   1   1   1   1      0   0   0   0   0   0   0   0   0
 -1  -1  -1  -1   1  1   1  -1  -1   1      0   0   0   0   0   0   0   0   0
 -1  -1  -1  -1   1  1  -1  -1  -1   1  …   1  -1   1  -1  -1  -1   1   1   1
 -1  -1  -1  -1   1  1   1   1  -1   1     -1   1   1  -1   1  -1  -1  -1  -1
 -1  -1  -1  -1   1  1   1  -1  -1   1     -1   1   1   1   1  -1   1  -1   1
  1   1   1  -1  -1  1   1   1   1   1      0   0   0   0   0   0   0   0   0
  1   1   1  -1  -1  1  -1   1   1   1      1  -1  -1  -1   1  -1  -1  -1  -1
  ⋮                  ⋮                  ⋱       ⋮                   ⋮      
 -1   1  -1  -1   1  1   1  -1   1   1  …   1  -1  -1  -1  -1   1  -1   1  -1
  1  -1   1  -1  -1  1   1   1  

In [29]:
V

64×71648 Matrix{Rational{Int64}}:
 1  1   1   1  1   1   1   1   1   1  …  1   1   1  1   1  1  1  1   1   1
 1  1   1   1  1   1   1   1   1  -1     0   1   0  0   0  0  1  0  -1   0
 1  1   1  -1  1   1  -1   1  -1  -1     0   0   0  0   0  0  1  0   0   0
 1  1   1   1  1   1   1   1   1   1     0  -1   0  0   0  0  0  0   1   0
 1  1   1   1  1   1   1  -1   1   1     1   0   1  1  -1  0  0  0   0   0
 1  1   1   1  1   1   1  -1   1  -1  …  0   0   0  0   0  0  0  0   0   0
 1  1   1  -1  1   1  -1  -1  -1  -1     0   0   0  0   0  0  0  0   0   0
 1  1   1   1  1   1   1  -1   1   1     0   0   0  0   0  0  1  0   0   0
 1  1  -1   1  1  -1   1  -1  -1   1     0   0   0  0   0  0  0  0   0  -1
 1  1  -1   1  1  -1   1  -1  -1  -1     0   0   0  0   0  0  0  0   0   0
 ⋮                 ⋮                  ⋱          ⋮               ⋮      
 1  1   1   1  1   1   1  -1   1   1  …  0   0   0  0   0  0  0  0   0   0
 1  1  -1   1  1  -1   1  -1  -1   1     1   0   0  1  -1  0  0  0  

In [30]:
CNC

64×71136 Matrix{Rational{Int64}}:
  1   1   1   1   1   1   1   1   1   1  …  1   1   1  1   1  1  1  1   1   1
  0   0  -1   0   0   0  -1  -1   1   1     0   1   0  0   0  0  1  0  -1   0
  0   0   0   0   0   0   0   0   0   0     0   0   0  0   0  0  1  0   0   0
  0   0   0   1   0   1   0   0   0   0     0  -1   0  0   0  0  0  0   1   0
  0   0   0   0   0  -1   0   0   1   0     1   0   1  1  -1  0  0  0   0   0
  0   0   0   0   0   0   0   0   1   0  …  0   0   0  0   0  0  0  0   0   0
  0   0   1   0   0   0   0   0   0   0     0   0   0  0   0  0  0  0   0   0
  0  -1   0   0   0  -1   0   0   0   0     0   0   0  0   0  0  1  0   0   0
  0   0   0  -1   0   0   0   0   0   0     0   0   0  0   0  0  0  0   0  -1
  0   0   0   0   0   0   0   0   0   0     0   0   0  0   0  0  0  0   0   0
  ⋮                   ⋮                  ⋱                 ⋮                ⋮
  0   0  -1   0   0   0   1   0   0   0  …  0   0   0  0   0  0  0  0   0   0
  0   0   0  -1   0   0   0   

# Tight facets for magic states

In [12]:
# Define magic state:
r = round((1/sqrt(2)),digits = 4);;
T = Vector{Number}([1,r,r,0]);;

In [13]:
# Create tensor of T states:
Ts = Vector{Vector{Number}}([T])
for n in 2:3
    Tn = T;;
    for i in 2:n
        Tn = kron(Tn,T)
    end
    push!(Ts,Tn)
end

In [17]:
T3 = Ts[3];;

In [22]:
V3 = transpose(stab3)*T3;; V3 = Vector{Float64}([round(z,digits=4) for z in V3]);

In [ ]:
tight_inequality_indices_t3 = findall(x->x==0,V3);;
tight_inequalities_t3 = transpose(stab3)[tight_inequality_indices_t3,:]

36×64 Matrix{Rational{Int64}}:
 1  0   0   0   0  -1   0  0   0   0  …   0   0  0   0  0  0   0   0   0   0
 1  0   0   0  -1   0   0  0   0   0      0  -1  0   0  0  0   0   0   0   0
 1  0   0   0   0   0  -1  0   0  -1      0   0  0   0  0  0   0   0   0   0
 1  0   0   0   0   0   0  0   0   0      0   0  0   0  0  0   0   0   0  -1
 1  0  -1   0   0   0   0  0   0   0      0   0  0   0  0  0  -1   0   1   0
 1  0   0   0   0   0   0  0   1   0  …   0   0  0   0  0  1   0   0   0   0
 1  0   0   0   0   0   0  0  -1   0      0   0  0   0  0  1   0   0   0   0
 1  0   0   0   0   0   0  0   0   0      0   0  0   0  0  0   0   0   0  -1
 1  0   0   0   0   0   0  0   0   0      0   0  0   0  0  0   0   0   0   1
 1  1   0   0   0   0   0  0   0   0      0   0  0   0  0  0  -1  -1   0   0
 ⋮                  ⋮                 ⋱       ⋮                ⋮          
 1  0   0   0   0   0  -1  0   0  -1     -1   0  0  -1  0  0   0   0   0   1
 1  0  -1   0   0   0   0  0   0   0      0   0

In [34]:
findall(x->x==zeros(Rational{Int64},36),tight_inequalities_t3*CNC)

CartesianIndex{2}[]

In [35]:
matrix = tight_inequalities_t3*CNC

36×71136 Matrix{Rational{Int64}}:
 0  0  2  2  0  1  0  4  0  0  0  2  0  …  2  2  0  1  2  2  0  0  2  0  0  2
 2  2  0  0  2  0  1  0  2  2  0  0  4     0  2  2  0  2  0  2  0  2  0  2  2
 0  0  2  0  0  2  0  0  2  2  0  0  0     1  2  0  2  2  0  4  1  4  0  2  2
 1  1  0  2  2  0  2  2  2  2  2  0  0     2  0  0  2  0  2  0  0  2  0  1  0
 2  2  0  0  1  2  1  0  0  4  0  0  0     2  2  2  2  0  1  0  0  0  0  2  0
 1  2  2  0  0  1  2  0  2  0  0  0  2  …  2  0  2  0  0  0  2  0  0  0  0  2
 1  1  0  2  0  0  2  2  2  2  0  0  0     0  0  0  2  0  2  0  0  2  0  1  0
 1  2  2  0  2  1  1  0  2  0  0  0  2     2  2  4  0  2  0  2  0  0  2  0  4
 1  2  0  2  0  1  1  0  2  2  0  0  2     0  0  0  2  0  0  2  0  2  0  2  0
 2  2  2  0  1  2  2  2  0  2  0  0  0     2  0  0  0  0  1  0  0  0  2  2  2
 ⋮              ⋮              ⋮        ⋱     ⋮              ⋮              ⋮
 0  1  0  0  0  2  1  0  2  2  0  0  0     2  2  2  1  2  2  4  0  2  2  1  0
 0  0  0  2  2  1  2  0  0  0 

In [50]:
tight_cncs = Matrix{Rational{Int64}}(undef,64,0)
for i in 1:size(matrix)[2]
    zero_elements = findall(x->x==0,matrix[:,i]);
    if length(zero_elements) ==36
        tight_cncs = hcat(tight_cncs,CNC[:,i])
    end
end

In [52]:
tight_cncs

64×72 Matrix{Rational{Int64}}:
 1   1  1   1   1   1  1   1  1  1  1  …  1   1   1  1  1   1   1   1  1   1
 0   0  0   0   0   0  0   0  0  0  0     0   0   0  0  0   0   0   0  0   0
 0   0  0   0   0   0  0   0  0  0  0     0   0   0  0  0   0   0   0  0   0
 0  -1  0   0   0   0  0  -1  0  0  0     0   0   0  0  0   1   0  -1  0   0
 0   0  0   0   0   0  0   0  0  0  0     0   0   0  0  0   0   0   0  0   0
 0   0  0   1   0   1  0   1  1  0  1  …  0   1   1  1  1   1   0   1  1   1
 1   0  1   0   0   1  1   1  0  1  0     1   1   1  0  0   1   1   1  0   0
 1   0  0   0   0  -1  0   0  0  0  0     0   1   1  0  1   0  -1   0  0   0
 0   0  0   0   0   0  0   0  0  0  0     0   0   0  0  0   0   0   0  0   0
 1   1  0   0   1   0  0   0  1  0  1     0   0   0  1  0   0   1   0  1   0
 ⋮                  ⋮               ⋮  ⋱             ⋮                 ⋮  
 0   0  0   0   0   0  0   0  0  0  0  …  0   0   0  0  0   0   0   0  0   0
 1   0  0   0   0   1  0   0  0  0  0     0   1

# 2-qubit

In [ ]:
# two qubit cnc operators
two_qubit_cncs = generate_all_cncs(2,[1,2]);
CNC2 = generate_cnc_vertices(2,two_qubit_cncs);

In [ ]:
# two qubit stabilizer states
two_qubit_maximal_isotropics = generate_all_cncs(2,[0]);
two_qubit_stabilizer_states = generate_cnc_vertices(2,two_qubit_maximal_isotropics);

In [61]:
# two qubit magic state:
T2 = Ts[2];

In [63]:
V2 = transpose(two_qubit_stabilizer_states)*T2;; V2 = Vector{Float64}([round(z,digits=4) for z in V2]);

tight_inequality_indices_t2 = findall(x->x==0,V2);;
tight_inequalities_t2 = transpose(two_qubit_stabilizer_states)[tight_inequality_indices_t2,:]

2×16 Matrix{Rational{Int64}}:
 1  0  0  0  0  -1   0  0  0   0  -1  0  0  0  0  -1
 1  0  0  0  0   0  -1  0  0  -1   0  0  0  0  0   1

In [64]:
two_qubit_cncs_tight_inequalities = tight_inequalities_t2*CNC2

2×432 Matrix{Rational{Int64}}:
 2  0  2  2  0  0  2  0  2  2  4  0  0  …  2  2  2  2  0  0  0  0  2  0  0  0
 0  2  2  0  2  2  0  0  2  0  0  2  2     0  2  2  2  0  0  2  2  0  0  0  4

In [ ]:
two_qubit_tight_cncs = Matrix{Rational{Int64}}(undef,16,0)
for i in 1:size(two_qubit_cncs_tight_inequalities)[2]
    zero_elements = findall(x->x==0,two_qubit_cncs_tight_inequalities[:,i]);
    if length(zero_elements) == 2
        two_qubit_tight_cncs = hcat(two_qubit_tight_cncs,CNC2[:,i])
    end
end

In [70]:
two_qubit_cncs_tight_inequalities

2×432 Matrix{Rational{Int64}}:
 2  0  2  2  0  0  2  0  2  2  4  0  0  …  2  2  2  2  0  0  0  0  2  0  0  0
 0  2  2  0  2  2  0  0  2  0  0  2  2     0  2  2  2  0  0  2  2  0  0  0  4

In [71]:
include("./libs/cnc.jl")

generate_cnc_vertices (generic function with 1 method)

In [73]:
cnc_4_1 = generate_canonical_cnc_with_n_m(4,1)

Maximal CNC set:
- Isotropic generators:
  ["XIII", "IIXI", "IXII"]
- Anticommuting Paulis:
  ["IXXX", "XIXY", "XIXZ"]


In [76]:
cnc_set_4_1 = find_full_set_for_given_cnc_set(cnc_4_1)

Set{Vector{Int64}} with 32 elements:
  [0, 1, 1, 1, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 0, 1]
  [1, 0, 0, 0, 0, 0, 0, 0]
  [1, 0, 1, 1, 0, 0, 0, 1]
  [1, 0, 1, 1, 0, 0, 0, 0]
  [0, 0, 0, 1, 0, 0, 0, 1]
  [0, 0, 0, 1, 0, 0, 0, 0]
  [1, 1, 0, 0, 0, 0, 0, 1]
  [1, 1, 0, 0, 0, 0, 0, 0]
  [0, 0, 1, 0, 0, 0, 0, 1]
  [0, 0, 1, 0, 0, 0, 0, 0]
  [1, 1, 1, 1, 0, 0, 0, 1]
  [1, 1, 1, 1, 0, 0, 0, 0]
  [0, 1, 0, 1, 0, 0, 0, 1]
  [0, 1, 0, 1, 0, 0, 0, 0]
  [1, 0, 0, 1, 0, 0, 0, 1]
  [0, 1, 1, 0, 0, 0, 0, 1]
  [0, 1, 1, 0, 0, 0, 0, 0]
  ⋮ 

In [75]:
SP_4 = SympPerm(4)

SympPerm(GAP: <permutation group with 2 generators>, Dict([0, 1, 1, 1, 0, 0, 0, 1] => 114, [0, 0, 1, 0, 0, 1, 0, 1] => 38, [0, 1, 1, 0, 1, 1, 0, 1] => 110, [1, 0, 0, 1, 1, 1, 0, 0] => 157, [1, 0, 0, 0, 0, 0, 1, 0] => 131, [0, 0, 0, 1, 0, 0, 0, 0] => 17, [0, 1, 0, 1, 0, 1, 1, 0] => 87, [1, 1, 0, 0, 1, 0, 1, 0] => 203, [0, 1, 0, 1, 1, 0, 0, 0] => 89, [0, 0, 0, 0, 1, 1, 0, 0] => 13…), Dict(56 => [0, 0, 1, 1, 0, 1, 1, 1], 35 => [0, 0, 1, 0, 0, 0, 1, 0], 110 => [0, 1, 1, 0, 1, 1, 0, 1], 60 => [0, 0, 1, 1, 1, 0, 1, 1], 220 => [1, 1, 0, 1, 1, 0, 1, 1], 30 => [0, 0, 0, 1, 1, 1, 0, 1], 6 => [0, 0, 0, 0, 0, 1, 0, 1], 67 => [0, 1, 0, 0, 0, 0, 1, 0], 234 => [1, 1, 1, 0, 1, 0, 0, 1], 215 => [1, 1, 0, 1, 0, 1, 1, 0]…))

In [84]:
cnc_orbit_4_1 = SympOrbit(4, SP_4, cnc_set_4_1)

SympOrbit(Set([[1, 0, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0]  …  [1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0]]), Set(Set{Vector{Int64}}[Set([[0, 0, 1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 0, 1, 1, 0], [1, 0, 0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 1, 0], [0, 0, 1, 0, 1, 0, 1, 1], [0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 1, 1], [1, 0, 1, 1, 0, 0, 1, 0]  …  [0, 0, 1, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 

In [82]:
using Pkg; Pkg.activate("./MyProject");;
using JLD2

@save "cnc_orbit_4_1.jld" cnc_orbit_4_1

  Activating project at `~/GitHub/LocalLambda/MyProject`


In [83]:
JLD2.load("./cnc_orbit_4_1.jld")

Dict{String, Any} with 1 entry:
  "cnc_orbit_4_1" => Set(Set{Vector{Int64}}[Set([[0, 0, 1, 0, 1, 1, 1, 0], [0, …

In [85]:
cnc_orbit_4_1.Canonical

Set{Vector{Int64}} with 32 elements:
  [1, 0, 1, 1, 1, 0, 0, 0]
  [0, 1, 1, 1, 0, 0, 0, 0]
  [0, 0, 0, 1, 1, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 0, 0]
  [1, 0, 1, 1, 0, 0, 0, 0]
  [1, 1, 0, 0, 1, 0, 0, 0]
  [0, 0, 1, 0, 1, 0, 0, 0]
  [0, 0, 0, 1, 0, 0, 0, 0]
  [1, 1, 1, 1, 1, 0, 0, 0]
  [0, 1, 0, 1, 1, 0, 0, 0]
  [1, 1, 0, 0, 0, 0, 0, 0]
  [0, 0, 1, 0, 0, 0, 0, 0]
  [1, 1, 1, 1, 0, 0, 0, 0]
  [1, 0, 0, 1, 1, 0, 0, 0]
  [0, 1, 1, 0, 1, 0, 0, 0]
  [0, 1, 0, 1, 0, 0, 0, 0]
  [1, 0, 1, 0, 1, 0, 0, 0]
  [0, 1, 1, 0, 0, 0, 0, 0]
  [1, 0, 0, 1, 0, 0, 0, 0]
  ⋮ 

In [86]:
cnc_4_1

Maximal CNC set:
- Isotropic generators:
  ["XIII", "IIXI", "IXII"]
- Anticommuting Paulis:
  ["IXXX", "XIXY", "XIXZ"]


In [91]:
collect(cnc_orbit_4_1.Orbit)[2]

Set{Vector{Int64}} with 32 elements:
  [1, 1, 0, 0, 1, 1, 1, 0]
  [1, 1, 1, 0, 0, 0, 1, 1]
  [0, 1, 1, 1, 0, 0, 0, 0]
  [1, 1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 0, 1]
  [1, 0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 0, 1]
  [1, 0, 1, 1, 0, 0, 0, 0]
  [1, 0, 1, 0, 1, 1, 0, 1]
  [0, 0, 0, 1, 0, 0, 1, 1]
  [1, 1, 0, 0, 0, 0, 0, 0]
  [1, 1, 0, 1, 1, 1, 0, 1]
  [1, 1, 1, 1, 0, 0, 0, 0]
  [0, 0, 0, 0, 1, 1, 1, 0]
  [0, 0, 1, 0, 0, 0, 1, 1]
  [1, 1, 1, 0, 1, 1, 0, 1]
  [0, 0, 1, 1, 1, 1, 1, 0]
  [0, 1, 0, 1, 0, 0, 1, 1]
  [0, 1, 0, 0, 1, 1, 1, 0]
  ⋮ 